This notebook takes the merged Yelp and Census Tract data frame and calculates the frequency of restaurant counts for each price range (including those Missing) for each Census tract. 
Using GEOID (census tract identifier), I merged household income distribution data for each census tract (in percentages), then calculate the actual number of households for each income range.


In [44]:
# Setting up modules
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
from shapely.geometry import Point
import matplotlib.pylab as plt

# Data path 
andrew_path = '/Users/andrewnorris/restaurant-scene-ads/'
data_path = "./data"
Yelp_BK_path = data_path +  "/Yelp/BK/"
Yelp_MN_path = data_path + "/Yelp/MN/"
path_census_tracts = './data/ACS/CensusTracts'

In [37]:
# Reading in Yelp shapefiles
BK_Yelp = gpd.read_file(Yelp_BK_path + "BK_Yelp_CensusTract_NTA.shp")
MN_Yelp = gpd.read_file(Yelp_MN_path + "MN_Yelp_CensusTract_NTA.shp")
MN_Yelp.head()

,id,alias,name,is_closed,review_cou,rating,price,categories,latitude,longitude,...,NTAName,0-25k,25-50k,50-75k,75-100k,100-125k,125-150k,> 150k,MeanLifeEx,geometry
0,ksksxd8J2SIs97ccPFV75A,brown-sugar-restaurant-new-york,Brown Sugar Restaurant,0,204,3.5,$$,"cuban,seafood,steak",40.869926,-73.915466,...,Marble Hill-Inwood,0.32,0.21,0.15,0.14,0.07,0.04,0.07,84.028571,POINT (-73.91546600000001 40.869926)
1,IHGm6huN_Z48KdorBjztSQ,guacamole-taqueria-new-york,Guacamole Taqueria,0,122,3.5,$$,mexican,40.869520,-73.916740,...,Marble Hill-Inwood,0.32,0.21,0.15,0.14,0.07,0.04,0.07,84.028571,POINT (-73.91674 40.86952)
2,rUBFgZU3QTk7IOgpccE8Og,indian-road-cafe-new-york,Indian Road Cafe,0,580,4.0,$$,"coffee,newamerican,bars",40.872915,-73.918441,...,Marble Hill-Inwood,0.32,0.21,0.15,0.14,0.07,0.04,0.07,84.028571,POINT (-73.91844082 40.87291516)
3,AflnoQBr01QmQIq5hVH-iA,la-essencia-restaurant-new-york,La Essencia Restaurant,0,13,3.5,$$,"mexican,dominican",40.870870,-73.915050,...,Marble Hill-Inwood,0.32,0.21,0.15,0.14,0.07,0.04,0.07,84.028571,POINT (-73.91504999999999 40.87087)
4,8o-B_1q4XB48CmdaXdm2KQ,raices-new-york,Raices,0,127,3.5,$$,"bars,latin",40.866330,-73.920160,...,Marble Hill-Inwood,0.32,0.21,0.15,0.14,0.07,0.04,0.07,84.028571,POINT (-73.92016 40.86633)


In [70]:
# Reading in NTA shapefile
NYC_neighborhood_tabn_areas = gpd.read_file(path_census_tracts + "/nynta_reproj.shp")
NYC_neighborhood_tabn_areas = NYC_neighborhood_tabn_areas.loc[:, ["NTACode", "NTAName", "geometry"]]
NYC_neighborhood_tabn_areas.head()

,NTACode,NTAName,geometry
0,BK88,Borough Park,POLYGON ((-73.97604935657382 40.63127590564677...
1,QN51,Murray Hill,POLYGON ((-73.80379022888246 40.77561011179247...
2,QN27,East Elmhurst,POLYGON ((-73.86109724401859 40.76366447708767...
3,QN07,Hollis,POLYGON ((-73.75725671509139 40.71813860166255...
4,MN06,Manhattanville,"POLYGON ((-73.94607828608069 40.8212632160616,..."


In [49]:
# Reading in csvs with distribution of income and life expectancy per NTA
BK_NTA_income_dist_life_exp = pd.read_csv(path_census_tracts + "/BK_NTA_LifeExp_Income_Dist.csv")
BK_NTA_income_dist_life_exp = BK_NTA_income_dist_life_exp.drop(["Unnamed: 0"], axis = 1)

MN_NTA_income_dist_life_exp = pd.read_csv(path_census_tracts + "/MN_NTA_LifeExp_Income_Dist.csv")
MN_NTA_income_dist_life_exp = MN_NTA_income_dist_life_exp.drop(["Unnamed: 0"], axis = 1)

In [38]:
# Coerce into a dataframe 
BK_Yelp_df = pd.DataFrame(BK_Yelp)
MN_Yelp_df = pd.DataFrame(MN_Yelp)

# Check colnames
#BK_Yelp_df.columns

In [30]:
# Treats dollar signs as a categorical variable and then count the number of restaurants for each price type in each NTA
# Counts the number of reviews 
BK_price_counts = pd.DataFrame(BK_Yelp_df.groupby(['NTACode', 'price'])['price'].count())
MN_price_counts = pd.DataFrame(MN_Yelp_df.groupby(["NTACode", "price"])["price"].count())
print(BK_price_counts.head(10))
print(MN_price_counts.head(10))

                 price
NTACode price         
BK09    $           29
        $$          46
        $$$          6
        MISSING     16
BK17    $           35
        $$          54
        $$$         10
        $$$$         2
        MISSING     21
BK19    $           22
                 price
NTACode price         
MN01    $           35
        $$          41
        MISSING     25
MN03    $           57
        $$          45
        $$$          1
        $$$$         1
        MISSING     36
MN04    $           50
        $$          31


In [31]:
# Unstack the grouped dataframe, fill all empty dollar signs with 0, reset index for both BK and MN
BK_price_counts = BK_price_counts.unstack(level='price', fill_value=0).reset_index()
MN_price_counts = MN_price_counts.unstack(level='price', fill_value=0).reset_index()

# Check
#print(BK_price_counts.head(10))

# Unstack, concat column names, rename for Brooklyn
BK_price_counts.columns = ['_'.join(col) for col in BK_price_counts.columns]
BK_price_counts.columns = ['NTACode','price_1', 'price_2', 'price_3', 'price_4','MISSING']

# Unstack, concat column names, rename for Manhattan
MN_price_counts.columns = ['_'.join(col) for col in MN_price_counts.columns]
MN_price_counts.columns = ['NTACode','price_1', 'price_2', 'price_3', 'price_4','MISSING']

# Price counts are currently in absolute numbers; this converts them into percentages for both boroughs
BK_price_counts_pct = BK_price_counts.copy()
BK_price_counts_pct.iloc[:, 1:] = BK_price_counts_pct.iloc[:, 1:].div(BK_price_counts_pct.iloc[:, 1:].sum(axis=1), axis=0)
MN_price_counts_pct = MN_price_counts.copy()
MN_price_counts_pct.iloc[:, 1:] = MN_price_counts_pct.iloc[:, 1:].div(MN_price_counts_pct.iloc[:, 1:].sum(axis=1), axis=0)

# Check
BK_price_counts_pct.head(10)

,NTACode,price_1,price_2,price_3,price_4,MISSING
0,BK09,0.298969,0.474227,0.061856,0.000000,0.164948
1,BK17,0.286885,0.442623,0.081967,0.016393,0.172131
2,BK19,0.379310,0.293103,0.068966,0.017241,0.241379
3,BK21,0.480000,0.240000,0.000000,0.000000,0.280000
4,BK23,0.375000,0.250000,0.125000,0.000000,0.250000
5,BK25,0.276190,0.390476,0.095238,0.009524,0.228571
6,BK26,0.531250,0.093750,0.000000,0.000000,0.375000
7,BK27,0.301887,0.415094,0.018868,0.000000,0.264151
8,BK28,0.441341,0.273743,0.005587,0.000000,0.279330
9,BK29,0.403846,0.278846,0.009615,0.009615,0.298077


So I tidied up Drew's code, commented and consolidated them into fewer chunks. The same operation has to be performed for reviews except we do not need to convert to percentages here.

In [73]:
# Create a grouped object for both MN and BK
BK_review_counts_grouped = BK_Yelp_df.groupby(['NTACode'])['review_cou']
MN_review_counts_grouped = MN_Yelp_df.groupby(['NTACode'])['review_cou']

# Calculate Q1, Q2, Q3 to get sense of distribution of the degree to which restaurants are reviewed in each NTA
BK_review_counts_Q123 = pd.DataFrame(BK_review_counts_grouped.quantile([0.25, 0.50, 0.75]))
MN_review_counts_Q123 = pd.DataFrame(MN_review_counts_grouped.quantile([0.25, 0.50, 0.75]))
# First unstack
BK_review_counts_Q123 = BK_review_counts_Q123.unstack().reset_index()
MN_review_counts_Q123 = MN_review_counts_Q123.unstack().reset_index()

# Unstack the grouped columns again
BK_review_counts_Q123.columns = ['_'.join(str(col)) for col in BK_review_counts_Q123.columns]
BK_review_counts_Q123.columns = ['NTACode','review_q1', 'review_q2', 'review_q3']
MN_review_counts_Q123.columns = ['_'.join(str(col)) for col in MN_review_counts_Q123.columns]
MN_review_counts_Q123.columns = ['NTACode','review_q1', 'review_q2', 'review_q3'] 

# If we are going to see the distribution of reviews across MN and BK, might as well join the two together
MN_BK_review_counts_Q123 = MN_review_counts_Q123.append(BK_review_counts_Q123)

# Add NTA geometry by merging 
MN_BK_review_counts_Q123 = NYC_neighborhood_tabn_areas.merge(MN_BK_review_counts_Q123, on = "NTACode")

# Export as shp
MN_BK_review_counts_Q123.to_file(data_path + "/Yelp/" + 'MN_BK_review_counts_Q123.shp', driver='ESRI Shapefile', encoding = 'utf-8')


# Check
#MN_review_counts_Q123.head()
#BK_review_counts_Q123.head()
#MN_BK_review_counts_Q123.shape
#MN_BK_review_counts_Q123.head()

Sorry Drew, but I think what you did below is problematic. What you is a dataframe of NTAs that contain information on the distribution of prices and income. Yes, your code managed to achieve that but geometry is wrong; it is tagged to each restaurant because you started with the Yelp data. 

As it stands, the Yelp data is a point shapefile with each point corresponding to the location of a restaurant. This is a very *long* dataset in the sense that for each restaurant, there is information on the income distribution and mean life expectancy of its NTA.  

Therefore, a much simpler way is to get the CSV of income distributions and life expectancies per NTA from the NYC Census Tracts notebook and then join. 

In [62]:
# Creating a df of the distribution of incomes, and prices, and mean life expectancy per NTA for both BK and MN
BK_NTA_income_dist_life_exp_price = BK_NTA_income_dist_life_exp.merge(BK_price_counts_pct, on = "NTACode")
MN_NTA_income_dist_life_exp_price = MN_NTA_income_dist_life_exp.merge(MN_price_counts_pct, on = "NTACode")

# Rename columns
BK_NTA_income_dist_life_exp_price = BK_NTA_income_dist_life_exp_price.rename(columns={"0-25k": "pct_0_25k", "25k-50k": "pct_25k_50k", '50k-75k':'pct_50k_75k', \
                                           '75k-100k':'pct_75k_100k', '100k-125k':'pct_100k_125k', '125k-150k':'pct_125k_150k',\
                                           '> 150k':'pct_> 150k'})
MN_NTA_income_dist_life_exp_price = MN_NTA_income_dist_life_exp_price.rename(columns={"0-25k": "pct_0_25k", "25-50k": "pct_25k_50k", '50-75k':'pct_50k_75k', \
                                           '75-100k':'pct_75k_100k', '100-125k':'pct_100k_125k', '125-150k':'pct_125k_150k',\
                                           '> 150k':'pct_> 150k'})
# Export as CSV
MN_NTA_income_dist_life_exp_price.to_csv(data_path + '/MN_NTA_income_dist_life_exp_price.csv')
BK_NTA_income_dist_life_exp_price.to_csv(data_path + '/BK_NTA_income_dist_life_exp_price.csv')

# Check
#BK_NTA_income_dist_life_exp_price.head()

,NTACode,NTAName,Households,pct_0_25k,pct_25k_50k,pct_50k_75k,pct_75k_100k,pct_100k_125k,pct_125k_150k,pct_> 150k,MeanLifeExp,price_1,price_2,price_3,price_4,MISSING
0,BK88,Borough Park,26710,0.331037,0.267802,0.132160,0.089292,0.063609,0.036541,0.079558,82.60000,0.330097,0.233010,0.067961,0.009709,0.359223
1,BK25,Homecrest,15528,0.257664,0.217800,0.153916,0.103941,0.107290,0.042375,0.117014,81.71875,0.276190,0.390476,0.095238,0.009524,0.228571
2,BK44,Madison,14900,0.214765,0.190805,0.166577,0.118658,0.100201,0.056711,0.152282,82.76000,0.474576,0.288136,0.033898,0.000000,0.203390
3,BK41,Kensington-Ocean Parkway,12228,0.243785,0.219496,0.156117,0.131420,0.096418,0.053566,0.099199,80.87500,0.492308,0.246154,0.030769,0.000000,0.230769
4,BK95,Erasmus,10137,0.271678,0.269212,0.163165,0.124593,0.061162,0.052086,0.058104,81.40000,0.450704,0.126761,0.000000,0.000000,0.422535


In [40]:
# # Rename columns
# BK_Yelp_df = BK_Yelp_df.rename(columns={"0-25k": "pct_0_25k", "25k-50k": "pct_25k_50k", '50k-75k':'pct_50k_75k', \
#                                            '75k-100k':'pct_75k_100k', '100k-125k':'pct_100k_125k', '125k-150k':'pct_125k_150k',\
#                                            '> 150k':'pct_> 150k'})
# MN_Yelp_df = MN_Yelp_df.rename(columns={"0-25k": "pct_0_25k", "25-50k": "pct_25k_50k", '50-75k':'pct_50k_75k', \
#                                            '75-100k':'pct_75k_100k', '100-125k':'pct_100k_125k', '125-150k':'pct_125k_150k',\
#                                            '> 150k':'pct_> 150k'})

In [30]:
# Cannot do this for MN because we do not have the total number of households 
# So I don't think we will need this chunk
#BK_Yelp_df['num_0_25k'] = BK_Yelp_df['Households']*BK_Yelp_df['pct_0_25k']
#BK_Yelp_df['num_25k_50k'] = BK_Yelp_df['Households']*BK_Yelp_df['pct_25k_50k']
#BK_Yelp_df['num_50k_75k'] = BK_Yelp_df['Households']*BK_Yelp_df['pct_50k_75k']
#BK_Yelp_df['num_75k_100k'] = BK_Yelp_df['Households']*BK_Yelp_df['pct_75k_100k']
#BK_Yelp_df['num_100k_125k'] = BK_Yelp_df['Households']*BK_Yelp_df['pct_100k_125k']
#BK_Yelp_df['num_> 150k'] = BK_Yelp_df['Households']*BK_Yelp_df['pct_> 150k']
#BK_Yelp_df.head()

,id,alias,name,is_closed,review_cou,rating,price,categories,latitude,longitude,...,pct_100k_125k,pct_125k_150k,pct_> 150k,geometry,num_0_25k,num_25k_50k,num_50k_75k,num_75k_100k,num_100k_125k,num_> 150k
0,6gzQLjzJk25ePm_JS7ZAug,esme-brooklyn-2,Esme,0,328,4.5,$$,newamerican|cocktailbars,40.733203,-73.954967,...,0.105529,0.086914,0.182642,POINT (-73.95496677 40.73320339),3248.0,2220.0,2464.0,2206.0,1712.0,2963.0
1,Swjm9no7DRqhThLlf0EHng,sama-street-brooklyn-2,Sama Street,0,58,4.5,$$,cocktailbars|panasian|tapasmallplates,40.732870,-73.954480,...,0.105529,0.086914,0.182642,POINT (-73.95448 40.73287),3248.0,2220.0,2464.0,2206.0,1712.0,2963.0
2,utM-5navObsVA5sCRHobzA,madre-brooklyn-2,Madre,0,38,5.0,MISSING,newamerican,40.733110,-73.957980,...,0.105529,0.086914,0.182642,POINT (-73.95798000000001 40.73311),3248.0,2220.0,2464.0,2206.0,1712.0,2963.0
3,L9SuMN3UsGipopWOe3pr9w,chiko-brooklyn-2,Chiko,0,36,5.0,MISSING,japanese|sushi,40.731900,-73.954220,...,0.105529,0.086914,0.182642,POINT (-73.95421999999998 40.7319),3248.0,2220.0,2464.0,2206.0,1712.0,2963.0
4,vyKBwzRdNX4yiJDIFv37iw,oxomoco-brooklyn-2,Oxomoco,0,247,4.0,$$$,mexican,40.729910,-73.955480,...,0.105529,0.086914,0.182642,POINT (-73.95548000000002 40.7299099),3248.0,2220.0,2464.0,2206.0,1712.0,2963.0


In [21]:
# bkntainc = BK_Yelp_df[['NTACode','NTAName', 'Households', 'pct_0-25k',
#        'pct_25k-50k', 'pct_50k-75k', 'pct_75k-100k', 'pct_100k-125k',
#        'pct_125k-150k', 'pct_> 150k', 'latitude','longitude','geometry', 'num_0-25k', 'num_25k-50k',
#        'num_50k-75k', 'num_75k-100k', 'num_100k-125k', 'num_125k-150k',
#        'num_> 150k']].copy()
# bkntainc.head()

,NTACode,NTAName,Households,pct_0-25k,pct_25k-50k,pct_50k-75k,pct_75k-100k,pct_100k-125k,pct_125k-150k,pct_> 150k,latitude,longitude,geometry,num_0-25k,num_25k-50k,num_50k-75k,num_75k-100k,num_100k-125k,num_125k-150k,num_> 150k
0,BK76,Greenpoint,16223,0.20021,0.136843,0.151883,0.13598,0.105529,0.086914,0.182642,40.733203,-73.954967,POINT (-73.95496677 40.73320339),3248.0,2220.0,2464.0,2206.0,1712.0,1410.0,2963.0
1,BK76,Greenpoint,16223,0.20021,0.136843,0.151883,0.13598,0.105529,0.086914,0.182642,40.732870,-73.954480,POINT (-73.95448 40.73287),3248.0,2220.0,2464.0,2206.0,1712.0,1410.0,2963.0
2,BK76,Greenpoint,16223,0.20021,0.136843,0.151883,0.13598,0.105529,0.086914,0.182642,40.733110,-73.957980,POINT (-73.95798000000001 40.73311),3248.0,2220.0,2464.0,2206.0,1712.0,1410.0,2963.0
3,BK76,Greenpoint,16223,0.20021,0.136843,0.151883,0.13598,0.105529,0.086914,0.182642,40.731900,-73.954220,POINT (-73.95421999999998 40.7319),3248.0,2220.0,2464.0,2206.0,1712.0,1410.0,2963.0
4,BK76,Greenpoint,16223,0.20021,0.136843,0.151883,0.13598,0.105529,0.086914,0.182642,40.729910,-73.955480,POINT (-73.95548000000002 40.7299099),3248.0,2220.0,2464.0,2206.0,1712.0,1410.0,2963.0


In [22]:
# bkntainc = bkntainc.drop_duplicates('NTACode')
# bkntainc.head()

,NTACode,NTAName,Households,pct_0-25k,pct_25k-50k,pct_50k-75k,pct_75k-100k,pct_100k-125k,pct_125k-150k,pct_> 150k,latitude,longitude,geometry,num_0-25k,num_25k-50k,num_50k-75k,num_75k-100k,num_100k-125k,num_125k-150k,num_> 150k
0,BK76,Greenpoint,16223,0.200210,0.136843,0.151883,0.135980,0.105529,0.086914,0.182642,40.733203,-73.954967,POINT (-73.95496677 40.73320339),3248.0,2220.0,2464.0,2206.0,1712.0,1410.0,2963.0
183,BK90,East Williamsburg,16075,0.284044,0.165723,0.123670,0.116579,0.087403,0.065070,0.157512,40.721030,-73.940740,POINT (-73.94074000000001 40.72103),4566.0,2664.0,1988.0,1874.0,1405.0,1046.0,2532.0
348,BK77,Bushwick North,19367,0.290029,0.207415,0.169515,0.106108,0.084164,0.053441,0.089327,40.707587,-73.933271,POINT (-73.93327099999998 40.707587),5617.0,4017.0,3283.0,2055.0,1630.0,1035.0,1730.0
602,BK78,Bushwick South,26616,0.349451,0.207582,0.152991,0.101743,0.068643,0.044222,0.075368,40.709139,-73.937227,POINT (-73.93722679999998 40.7091391),9301.0,5525.0,4072.0,2708.0,1827.0,1177.0,2006.0
738,BK73,North Side-South Side,23862,0.192566,0.167337,0.113654,0.105859,0.088718,0.070321,0.261546,40.721189,-73.957000,POINT (-73.95700045 40.72118902),4595.0,3993.0,2712.0,2526.0,2117.0,1678.0,6241.0


In [23]:
# This is problematic because this is now a point shapefile. Hence, it was corrupted somewhere down the line becase
# NTA shapefiles should be polgons, not points
#bk_price_inc = BK_price_counts.merge(bkntainc, how='inner', on='NTACode')
#bk_price_inc.head()

,NTACode,price_1,price_2,price_3,price_4,MISSING,NTAName,Households,pct_0-25k,pct_25k-50k,...,latitude,longitude,geometry,num_0-25k,num_25k-50k,num_50k-75k,num_75k-100k,num_100k-125k,num_125k-150k,num_> 150k
0,BK09,29,46,6,0,16,Brooklyn Heights-Cobble Hill,11115,0.115070,0.108052,...,40.699268,-73.992311,POINT (-73.99231090000001 40.69926820000001),1279.0,1201.0,777.0,1231.0,1042.0,840.0,4745.0
1,BK17,35,54,10,2,21,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,26150,0.253805,0.202600,...,40.577090,-73.952661,POINT (-73.95266059999999 40.57709038),6637.0,5298.0,3965.0,2832.0,2384.0,1692.0,3342.0
2,BK19,22,17,4,1,14,Brighton Beach,14557,0.395823,0.185272,...,40.576720,-73.964270,POINT (-73.96427 40.57672),5762.0,2697.0,2311.0,990.0,681.0,521.0,1595.0
3,BK21,24,12,0,0,14,Seagate-Coney Island,11236,0.478907,0.203364,...,40.578805,-73.983811,POINT (-73.983811 40.578805),5381.0,2285.0,1447.0,711.0,642.0,240.0,530.0
4,BK23,6,4,2,0,4,West Brighton,8401,0.377217,0.213070,...,40.579194,-73.980654,POINT (-73.9806538 40.5791945),3169.0,1790.0,1250.0,962.0,428.0,266.0,536.0


In [24]:
#bk_price_inc.shape

(51, 25)

In [25]:
#bk_price_inc.to_csv(data_path + '/BK_YelpPriceFreq_NTAIncDist.csv')


In [26]:
# # Creating another csv that has the yelp price rating broken down by percentage
# bk_pct_price_inc = bk_counts_pct.merge(bkntainc, how='inner', on='NTACode')
# bk_pct_price_inc.to_csv(data_path + '/BK_YelpPriceFreqPct_NTAIncDist.csv')
# bk_pct_price_inc.shape

(51, 25)